In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
company = pd.read_parquet('/home/group3/group3/Finalized/data/company_overview_cleaned.parquet')
glassd_classification = pd.read_parquet('/home/group3/group3/Finalized/data/glassd_classification_cleaned.parquet')
text = pd.read_parquet('../../../glassdoor/glassd_review_internal_data_2024-03-19.parquet')
# jobtitle_mapping = pd.read_parquet('../../../jobtitle_mapping_2024-03-19.parquet')
# location_mapping = pd.read_parquet('../../../location_mapping_2024-03-19.parquet')
USstock_daily = pd.read_parquet('/quant_data/schedule_store/data/daily_US_Stock/crsp_dsf_20240620.pq')

In [3]:
company2 = pd.read_parquet('/home/group3/glassdoor/company_overview_2024-03-19.parquet')
glassd2 = pd.read_parquet('/home/group3/glassdoor/glassd_classification_2024-03-19.parquet')

In [7]:
glassd2['glassdoorId'].nunique()

4736

In [6]:
glassd_classification['glassdoorId'].nunique()

4325

## Stock filtering (by company)

### 1. Trading between 2020-2023

In [4]:
start_trading = USstock_daily[['ticker','date']].groupby('ticker').min()

In [5]:
end_trading = USstock_daily[['ticker','date']].groupby('ticker').max()

In [6]:
trading_2023_req = start_trading[(start_trading['date'] < '20200101') & (end_trading['date'] > '20221130')]

In [7]:
trading_2023_req

,date
ticker,
A,20000103
AA,20000103
AAL,20131209
AAMC,20130912
AAME,20000103
...,...
ZTS,20130201
ZUMZ,20050506
ZUO,20180412


### 2. at least 5 review

In [7]:
glassd_company_merge = pd.merge(glassd_classification, company, on='glassdoorId')

In [9]:
review_count_per_company = glassd_company_merge[['ticker','reviewId']].groupby('ticker').count()

In [57]:
filter_g5 = review_count_per_company[review_count_per_company['reviewId'] > 5]

In [106]:
filter_g50 = review_count_per_company[review_count_per_company['reviewId'] > 50]

### 3. first review happens before 2013

In [59]:
earliest_review = glassd_company_merge[['ticker','reviewDateTime']].groupby('ticker').min()

In [60]:
filter_earliest_review = earliest_review[earliest_review['reviewDateTime'] < '2013-01-01']

In [61]:
filter_earliest_review

,reviewDateTime
ticker,
1,2010-10-25 19:35:02
A,2008-05-01 08:30:51
AA,2008-06-12 18:37:55
AAL,2008-06-11 06:23:31
AAOI,2010-06-30 10:29:20
...,...
ZUMZ,2008-06-16 09:03:52
ZUO,2011-01-25 14:16:02
ZVO,2008-09-23 22:16:20


### 4. filter market cap

In [13]:
mcap_per_comp = USstock_daily[USstock_daily['date']>'20130101'][['mcap', 'ticker']].groupby('ticker').mean()

In [18]:
filters['mcap>e9'] = list(mcap_per_comp[mcap_per_comp['mcap']>1e9].index.values)

In [19]:
filters['mcap>e10'] = list(mcap_per_comp[mcap_per_comp['mcap']>1e10].index.values)

# Grouping porfolio by different criteria

In [90]:
by_industry = company[['industryName', 'ticker']]

In [92]:
by_industry = by_industry[by_industry['industryName']!='-']

In [96]:
by_industry_count = by_industry.groupby('industryName').count()
gt5_perindustry = by_industry_count[by_industry_count['ticker'] > 5].index

In [111]:
gt5_perindustry = company[company['industryName'].isin(gt5_perindustry)]['ticker'].values

In [ ]:
by_revenue = company[['revenue_mapping', 'ticker']].dropna()

In [82]:
mapping = {0.0: '>1B', 
           1.0: '100M-1B', 
           2.0: '1M-100M', 
           3.0: '<1M'
}

by_revenue['revenue_bin'] = by_revenue['revenue_mapping'].map(mapping)

In [86]:
by_revenue = by_revenue[['revenue_bin', 'ticker']].reset_index(drop=True)

In [73]:
by_employee = company[['size', 'ticker']].dropna()
by_employee.rename(columns={'size':'group'},inplace=True)

### time series rolling review counts

In [117]:
glassd_classification['reviewDateTime']=pd.to_datetime(glassd_classification['reviewDateTime'])
glassd_classification.set_index('reviewDateTime',inplace=True,drop=True)
monthly_review_count = glassd_classification[['reviewId','ticker']].groupby('ticker').resample('M').count()
avg_monthly_review_count = monthly_review_count['reviewId'].reset_index()[['ticker','reviewId']].groupby('ticker').mean()
avg_monthly_review_count = avg_monthly_review_count.sort_values('reviewId')
month_g1 = avg_monthly_review_count[avg_monthly_review_count['reviewId']>1]
month_g10 = avg_monthly_review_count[avg_monthly_review_count['reviewId']>10]

quarterly_review_count = glassd_classification[['reviewId','ticker']].groupby('ticker').resample('Q').count()
avg_quarterly_review_count = quarterly_review_count['reviewId'].reset_index()[['ticker','reviewId']].groupby('ticker').mean()
avg_quarterly_review_count = avg_quarterly_review_count.sort_values('reviewId')

quarter_g1 = avg_quarterly_review_count[avg_quarterly_review_count['reviewId']>1]
quarter_g10 = avg_quarterly_review_count[avg_quarterly_review_count['reviewId']>10]
quarter_g100 = avg_quarterly_review_count[avg_quarterly_review_count['reviewId']>100]

/tmp/ipykernel_3205583/1110825391.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_review_count = glassd_classification[['reviewId','ticker']].groupby('ticker').resample('M').count()
/tmp/ipykernel_3205583/1110825391.py:9: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  quarterly_review_count = glassd_classification[['reviewId','ticker']].groupby('ticker').resample('Q').count()


In [120]:
all_filters = {}
all_filters['trade_between_2020_2023'] = list(trading_2023_req.index.values)
all_filters['more_than_5_reviews'] = list(filter_g5.index.values)
all_filters['more_than_50_reviews'] = list(filter_g50.index.values)
all_filters['on_glassdoor_before_2013'] = list(filter_earliest_review.index.values)
all_filters['more_than_5_companies_per_industry'] = list(gt5_perindustry)
all_filters['more_than_1_review_per_month'] = list(month_g1.index.values)
all_filters['more_than_10_review_per_month'] = list(month_g10.index.values)
all_filters['more_than_1_review_per_quarter'] = list(quarter_g1.index.values)
all_filters['more_than_10_review_per_quarter'] = list(quarter_g10.index.values)
all_filters['more_than_109_review_per_quarter'] = list(quarter_g100.index.values)

In [122]:
import pickle
with open('filters.pkl', 'wb') as file:
    pickle.dump(all_filters, file)

In [125]:
by_industry

,industryName,ticker
0,Healthcare Services & Hospitals,AAC
1,Machinery Manufacturing,AAON
2,Aerospace & Defence,AIR
3,Consumer Product Manufacturing,ACCO
4,Banking & Lending,ACNB
...,...,...
4496,Energy & Utilities,RDC
4497,Healthcare Services & Hospitals,STE
4498,Biotech & Pharmaceuticals,PRGO
4499,Machinery Manufacturing,SSYS


In [126]:
by_revenue.rename(columns={'revenue_bin':'group'},inplace=True)
by_industry.rename(columns={'industryName': 'group'},inplace=True)

In [129]:
by_employee.to_csv('by_employee.csv',index=False)
by_industry.to_csv('by_industry.csv',index=False)
by_revenue.to_csv('by_revenue.csv',index=False)

## Preliminary cleaning

In [23]:
# 1. drop rows with more unhelpful comments than helpfuls
glassd_classification_cleaned = glassd_classification[(glassd_classification['countNotHelpful']-glassd_classification['countHelpful'])<10]

# 2. drop rows with empty pros/cons
glassd_classification_cleaned = glassd_classification_cleaned[~glassd_classification_cleaned['reviewId'].isin(text[text[['pros','cons']].isna().any(axis=1)]['reviewId'])]

# 3. drop duplicates
glassd_classification_cleaned = glassd_classification_cleaned.drop_duplicates(subset=['reviewId'])

# 4. drop unnecessary columns

glassd_classification_cleaned = glassd_classification_cleaned.drop(['Id','dataVendorId','reviewDetailUrl','updateDateTime'],axis=1)


In [19]:
text[text['reviewId'].isin([1035568])]

,reviewId,summary,pros,cons,advice,other_data,updateDateTime
98188,1035568,cool,Facebook during work hours. so fun,boring no duties for intern,None,"{""isLegal"": true, ""employer"": {""__ref"": ""Emplo...",2023-08-28 01:56:59


In [7]:
glassd_classification.isna().sum()

Id                                     0
dataVendorId                           0
reviewId                               0
glassdoorId                            0
reviewDateTime                         0
reviewDetailUrl                     1955
isCurrentJob                           0
lengthOfEmployment                     0
jobtitle_ref                     1011779
location_ref                     1667847
ratingOverall                          0
ratingCareerOpportunities              0
ratingCompensationAndBenefits          0
ratingCultureAndValues                 0
ratingSeniorLeadership                 0
ratingWorkLifeBalance                  0
ratingDiversityAndInclusion            0
ratingBusinessOutlook            1506386
ratingCeo                        1508890
ratingRecommendToFriend          1233076
countHelpful                           0
countNotHelpful                        0
updateDateTime                         0
dtype: int64

In [27]:
revenue_mapping = {
    '$10+ billion (USD)': 0,
    '$5 to $10 billion (USD)': 0,
    '$2 to $5 billion (USD)': 0,
    '$1 to $2 billion (USD)': 0,
    '$1 to $5 billion (USD)': 0,
    '$500 million to $1 billion (USD)': 1,
    '$100 to $500 million (USD)': 1,
    '$10 to $25 million (USD)': 2,
    '$5 to $25 million (USD)': 2,
    '$5 to $10 million (USD)': 2,
    '$1 to $5 million (USD)': 2,
    'Less than $1 million (USD)': 3,
    'Unknown  Non-Applicable': None,
    'Unknown / Non-Applicable': None,
    'Company - Public': None,
    'SEAC': None,
}

company['revenue_mapping'] = company['revenue'].map(revenue_mapping)

In [29]:
company.isna().sum()


compustatName         0
cusip                 0
shortName             1
website              97
comType              58
revenue              57
headquarters         69
size                 57
stock              1471
industryName         58
glassdoorId           0
yearFounded        1516
reviewCount        4512
revenue_mapping    1828
dtype: int64

In [63]:
# missing ticker mappings
company_name_mapping = USstock_daily[['comnam','ticker']].drop_duplicates(subset=['comnam'])
company_name_mapping.dropna(inplace=True)
company_name_mapping.index = company_name_mapping['comnam']
company_name_mapping = company_name_mapping.drop('comnam',axis=1)

company['compustatName'] = company['compustatName'].apply(lambda x: x.strip())
company['ticker_'] = company['compustatName'].map(company_name_mapping['ticker'])
company['ticker'] = company.apply(lambda x: x['stock'] if x['stock']!=None else x['ticker_'],axis=1)

In [66]:
company.isna().sum()

compustatName         0
cusip                 0
shortName             1
website              97
comType              58
revenue              57
headquarters         69
size                 57
stock              1471
industryName         58
glassdoorId           0
yearFounded        1516
reviewCount        4512
revenue_mapping    1828
ticker              454
ticker_            1607
dtype: int64

In [75]:
# drop any remaining tickers with no valid stock ticker
company = company[~company[['ticker']].isna().any(axis=1)]

In [ ]:
# country mapping
company[['city','state/country']] = company['headquarters'].str.split(',', expand=True)
company['country'] = company['state/country'].apply(lambda x: 'US' if isinstance(x,str) and (len(x.strip())==2 or x=='United States') else x)

In [95]:
# drop unnecessary columns

company = company.drop(['cusip','ticker_', 'stock'],axis=1)


In [6]:
stock_monthly = pd.read_csv('/home/group3/group3/Finalized/data/stock_monthly.csv')
stock_monthly['date'] = pd.to_datetime(stock_monthly['date'])

In [20]:
len(stock_monthly[stock_monthly['ret']>3]['ticker'].unique())

202

In [22]:
len(set(stock_monthly[stock_monthly['ret']<0.2]['ticker'].unique()))

80

In [16]:
filters = pd.read_pickle('/home/group3/group3/Finalized/groupings/filters.pkl')


In [25]:
filters['extreme_stocks'] = list(set(stock_monthly[stock_monthly['ret']>3]['ticker'].unique()).union(set(stock_monthly[stock_monthly['ret']<0.2]['ticker'].unique())))

In [24]:
import pickle
with open('filters.pkl', 'wb') as file:
    pickle.dump(filters, file)